### LangChain + Deep Lake + ChatGPT-3.5 快速学习github开源代码 

#### 1. dependences
##### 1.1 deeplake - online vectors DB
##### 1.2 tiktoken - convert the text to vectors

In [2]:
%pip install --upgrade langchain deeplake openai tiktoken

.........botocore<1.32.0,>=1.31.30->boto3->deeplake) (1.16.0)



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from dotenv import load_dotenv
import os
os.environ.clear()
load_dotenv()
api_key = os.environ['OPENAI_API_KEY']
token = os.environ['ACTIVELOOP_TOKEN']

In [32]:

embeddings = OpenAIEmbeddings(disallowed_special=())

In [34]:
!git clone https://github.com/AntonOsika/gpt-engineer.git

Cloning into 'gpt-engineer'...


In [33]:
from langchain.document_loaders import TextLoader

root_dir = '../brian_jupyter_lab/gpt-engineer/'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try:
            loader = TextLoader(os.path.join(dirpath, file),encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e:
            pass


In [25]:
len(docs)

154

In [34]:
from langchain.text_splitter import CharacterTextSplitter
# 对文本切块，8000的字符切一次，每个切块的上下文件有100个字符的重叠
text_splitter = CharacterTextSplitter(chunk_size=8000, chunk_overlap=100)
texts= text_splitter.split_documents(docs)

Created a chunk of size 1540, which is longer than the specified 1500


In [36]:
len(texts)

154

In [41]:
username="huangliang"
db = DeepLake(dataset_path=f"hub://{username}/gpt_enginner_02", embedding=embeddings,token=token)
db.add_documents(texts)

Deep Lake Dataset in hub://huangliang/bjl_gpt_enginner_source already exists, loading from the storage


Dataset(path='hub://huangliang/bjl_gpt_enginner_source', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
 embedding  embedding  (425, 1536)  float32   None   
    id        text      (425, 1)      str     None   
 metadata     json      (425, 1)      str     None   
   text       text      (425, 1)      str     None   


['50bb68a7-3f30-11ee-b60b-dc215ccacab9',
 '50bb68a8-3f30-11ee-98a7-dc215ccacab9',
 '50bb68a9-3f30-11ee-b082-dc215ccacab9',
 '50bb68aa-3f30-11ee-b8f9-dc215ccacab9',
 '50bb68ab-3f30-11ee-9aea-dc215ccacab9',
 。。。。。。。。,
 '50bca165-3f30-11ee-ba3d-dc215ccacab9']

In [42]:
db = DeepLake(dataset_path='hub://huangliang/gpt_enginner_02', read_only=True,embedding=embeddings,token=token)

Deep Lake Dataset in hub://huangliang/gpt_enginner_02 already exists, loading from the storage


In [43]:
retriever=db.as_retriever()
#distance_metric计算两个的点的距离
retriever.search_kwargs['distance_metric'] = 'cos'
#fetch_k 从文档中要检索的数据点的数据
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
#k 返回相似度最高的向量的数量  
retriever.search_kwargs['k'] = 10

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model="gpt-3.5-turbo-16k",openai_api_key=api_key,temperature=0.1)
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [45]:
questions = [
    "what dose gpt-engineer work?",
    "How to use gpt-engineer?"
]
chat_history = []

for question in questions:
    result = qa({"question":question,"chat_history":chat_history})
    chat_history.append((question,result['answer']))
    print(f"Q:\n {question} \n")
    print(f"A:\n {result['answer']} \n")

Q:
 what dose gpt-engineer work? 

A:
 GPT-Engineer is a project that uses OpenAI's GPT-4 model to automate various software engineering tasks. It provides a conversational interface to interact with the GPT-4 model and guide it to generate code, clarify instructions, generate specifications, and more. The project is designed to be easy to use, even for users without a background in coding. It leverages the capabilities of GPT-4 to automate software engineering processes and make development tasks more efficient. 

Q:
 How to use gpt-engineer? 

A:
 To use gpt-engineer, you can follow these general steps:

1. Install gpt-engineer: You can install gpt-engineer by running `pip install gpt-engineer` or by cloning the GitHub repository and installing it from the source code.

2. Set up your project: Create a new folder for your project and copy the example project files into it. Fill in the `prompt` file with your desired prompt or instructions.

3. Set your OpenAI API key: Export your Ope

In [15]:
def ask(question,chat_history):
    res = qa({"question":question,"chat_history":chat_history})
    print(f"Q:\n {question} \n")
    print(f"A:\n {res['answer']} \n")

In [16]:
ask("What's the main programming language used in gpt-engineer?",chat_history)

Q:
 What's the main programming language used in gpt-engineer? 

A:
 The primary programming language used in GPT-Engineer is Python. 



In [17]:
ask("What's python class to generate the prompt in gpt-engineer",chat_history)

Q:
 What's python class to generate the prompt in gpt-engineer 

A:
 The Python class used to generate prompts in GPT-Engineer is the `AI` class. It serves as the main interface to the GPT-4 model and provides methods to start a conversation with the model, continue an existing conversation, and format system and user messages. 



In [18]:
ask("Can tell me the file in 'preprompts' floder of gpt-engineer?",chat_history)

Q:
 Can tell me the file in 'preprompts' floder of gpt-engineer? 

A:
 In the 'preprompts' folder of GPT-Engineer, you can find the following files:

1. fix_code
2. generate
3. philosophy
4. qa
5. respec
6. spec
7. unit_tests
8. use_feedback

These files contain predefined prompts that guide the AI in performing different tasks. 



In [19]:
ask("Tell me the order of gpt-engineer use the prompt file in 'preprompts' floder?",chat_history)

Q:
 Tell me the order of gpt-engineer use the prompt file in 'preprompts' floder? 

A:
 The order of using the prompt file in the 'preprompts' folder for GPT-Engineer is as follows:

1. Fix Code (`gpt_engineer/preprompts/fix_code`)
2. Generate (`gpt_engineer/preprompts/generate`)
3. Philosophy (`gpt_engineer/preprompts/philosophy`)
4. QA (`gpt_engineer/preprompts/qa`)
5. Respec (`gpt_engineer/preprompts/respec`)
6. Spec (`gpt_engineer/preprompts/spec`)
7. Unit Tests (`gpt_engineer/preprompts/unit_tests`)
8. Use Feedback (`gpt_engineer/preprompts/use_feedback`)

This order represents a sequence of different tasks that the AI can perform based on the instructions provided in each prompt. It starts with fixing code, followed by generating code, considering best practices, seeking clarification, reviewing specifications, writing unit tests, and utilizing user feedback for code generation. 



In [66]:
ask("Tell me the Python class of gpt-engineer use the prompt file in 'preprompts' floder?",chat_history)

Q:
 Tell me the Python class of gpt-engineer use the prompt file in 'preprompts' floder? 

A:
 The Python class used by gpt-engineer to access the prompt file in the 'preprompts' folder is the `DB` class. 



In [67]:
ask("Tell me what dose DB.py do in gpt-engineer ?",chat_history)

Q:
 Tell me what dose DB.py do in gpt-engineer ? 

A:
 The purpose of `DB.py` in GPT-Engineer is to represent a simple database that stores its data as files in a directory. It serves as a key-value store, where keys are filenames and values are file contents. The `DB` class provides methods to check if a key (filename) exists in the database, get the value (file content) associated with a key, and set the value associated with a key. The `DB` class is responsible for managing the data used by the GPT-Engineer system. 



In [68]:
ask("Tell me Where dose DB.py store the data in gpt-engineer ?",chat_history)

Q:
 Tell me Where dose DB.py store the data in gpt-engineer ? 

A:
 The `DB.py` file in `gpt-engineer` represents a simple database that stores its data as files in a directory. The directory where the data is stored can be specified when creating an instance of the `DB` class by providing a path argument. If the directory does not already exist, the `DB` class will create it. Each file in the directory represents a key-value pair, where the filename serves as the key and the file content serves as the value. 

